## Setting up a hydration free Energy calculuation
This notebook will guide you through how to use FESetup in order to run an alchemical free energy calculation.    


The notebook forms part of the CCPBio-Sim workshop **Tackling Alchemistry with FESetup and Sire SOMD** run on the 10th of April 2018 at the University of Bristol.

Author: Anotnia Mey   
Email: antonia.mey@ed.ac.uk

### Setting up the notebook with all necessary imports

In [1]:
%pylab inline
import nglview

Populating the interactive namespace from numpy and matplotlib


### Setting the stage
We want to compute the relative free energy of hydration between ethane and methanol. That is the free energy difference between hydrating an ethane molecule in water and a methanol molecule in water. It assumes that you had a look at the slides of lecture 1 or attended lecture1 (maybe youtube link here).

Below you can see a thermodynamic cycle for the relative hydration free energy of ethane and methanol:
![therm_cycle](Data/Therm_cycle.jpg)

We use FESetup to authomatically generate the perturbation information needed for running the alchemical free energy calcualtions

### Visualise molecule

Have a look in the FESetup directory. You will find a file called `morph.in` and a directory called `solute`. `morph.in` is the main input file and we will look at it more closely in a second. `solute` contains two further directories: ethane and methanol each containing a pdb structure file called `solute.pdb` containing the 3D structure of the two molecules we want to compute the relative free energy difference between. 



In [3]:
import MDAnalysis as mda
import nglview as nv
from nglview.datafiles import PDB, XTC

u = mda.Universe('FESetup/solutes/ethane/solute.pdb')

protein = u.select_atoms('all')

In [4]:
w = nv.show_mdanalysis(protein)
w

A Jupyter Widget

### FESetup
You can open a terminal and run FESetup from the command line by simply typing:   
    ```FESetup --help```

Or you can run things within the notebook. In order to access command line arguments in a notebook as was covered before you can use the exclamation mark as an escape.

In [23]:
%%capture
!/Users/toni_brain/Software/FESetup1.1/bin/FESetup --help

FESetup is a command line tool and you can pass it an input file defining what you would like to set up. Here we are wokring with ethane and methanol. Let's have a closer look at morph.in



In [34]:
!head -n 7 FESetup/morph_short.in

[globals]
logfile = eth-meth.log
forcefield = amber, ff14SB, tip3p, hfe
gaff = gaff2
mdengine = amber, sander 
AFE.type = Sire
AFE.separate_vdw_elec = false


The first section consists of global directives. Important here are the two directives `AFE.tye=Sire` and `AFE.separate_vdw_elec = false`. The first one means that Sire compatible output is generated and the second one means that the simulation will be carried out using a single step perturbation, perturbing charges and van der Waals interactions at the same time.

In [40]:
!tail -n 19 FESetup/morph.in

[ligand]
basedir = solute 
file.name = solute.pdb
molecules = ethane, methanol

# the following are required to create the morph in solution
box.type = rectangular
box.length = 12.0
neutralize = yes
min.nsteps = 100

min.ncyc = 100
min.restr_force = 10.0
min.restraint = notsolvent

#morph_pairs defines how the morphing between two molecules is done
#The example below morphs ethane to methanol
morph_pairs = ethane > methanol



For FESetup to understand what files to work on a strict directory structure is used. `basdir` gives information where it can find all molecules files. Here the basedir is called `solute`. In `solute` you have the molecules ethane and methanol as indicated by the corresponding directory and the actual filename which should be the same for all solutes is in this case `file.name = solute.pdb`. `molecules` defines the molecules that should actually be setup and in this case is the list of directories.

Any directive that starts with `box.type` indicates that a solvated version of the molecule should be gnerated. FESetup also allows to run a minimisation straight away by using the different `min` directives. In the above example a minimisation for 100 steps using a restraint on non-solvent molecules of 10. 

To execute FESetup you either type into your terminal `FESetup morph.in` or exectute the next cell.

In [17]:
!FESetup morph.in

The output should look like this:
```bash
FESetup morph.in 

=== FESetup release 1.2.1, SUI version: 0.8.3 ===

Please cite: HH Loeffler, J Michel, C Woods, J Chem Inf Mod, 55, 2485
             DOI 10.1021/acs.jcim.5b00368
For more information please visit http://www.ccpbiosim.ac.uk/fesetup/

Making ligand ethane...
Making ligand methanol...
Morphs will be generated for Sire
Morphing ethane to methanol...

=== All molecules built successfully ===


real	0m7.047s
user	0m6.712s
sys	0m0.349s
```

Furthermore  some additional files have been generated

In [44]:
ls

01_FESetup.ipynb     Func.jpg             ethane~methanol.pdf
Data/                HydrationF.jpg       gradient.jpg
FESetup/             Therm_cycle.key      menu.html
FESetup.zip          Therm_cycle.pdf      methanol.pdb
FUNC_0.in            both_pmf.jpg         pmf.jpg
FUNC_1.in            config_free          pmf.png
Free_en.jpg          config_vacuum        script
Free_en.png          ethane.pdb
Free_en_corr.jpg     ethane~methanol.jpg


Directories with an underscore `_` are output directories that contain the relevant information for the simualtion input. The most useful file is the generated logfile `eth-meth.log`. It contains all amber commands that were run behind the scenes. 

### Solvated ethane

In [ ]:
#visualise ethane here

### Solvated methanol

In [ ]:
## visualise solvated methanol fill in code to visualise solvated methanol

In [ ]:
## Widget question box

In [5]:
## perturbation files

## Exercises

1. Run with equilibration (error)
2. How to compute perturbation methanol -> ethane
3. Advanced (Setup for methanol -> ethanol using only minimisation input)
4. Advanced run some aspects of the simulation in vaccuum 
5. Atom mapping